In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

symbols = pd.read_csv("/Users/jochien/Documents/MaxRereach/strategy/crypto_ohlc_day/symbols.csv").symbol.tolist()
crypto_prices = ((pd.concat([pd.read_csv(f"/Users/jochien/Documents/MaxRereach/strategy/crypto_ohlc_day/{symbol}.csv", index_col='date', parse_dates=True)
        ['close'].rename(symbol)for symbol in symbols],axis=1,sort=True)))
df = crypto_prices.loc[:,~crypto_prices.columns.duplicated()]

In [2]:
ret_df = df.pct_change().dropna(axis=0)
final_output = {}
alpha_list = []
beta_list = []
ERi_list = []

In [3]:
for symbol in ret_df.columns[1:]:
    X = ret_df[['BTCUSDT']].assign(Intercept=1)
    Y = ret_df[symbol]
    X1 = sm.add_constant(X)     
    model = sm.OLS(Y, X1)
    results = model.fit()
    alpha = pd.read_html(results.summary().as_html(), header=0, index_col=0)[1].iloc[1][0]
    beta = pd.read_html(results.summary().as_html(), header=0, index_col=0)[1].iloc[0][0]
    ERi = (alpha + beta*ret_df['BTCUSDT'].mean())*365 # alpha + beta * rm
    final_output[symbol] = [alpha, beta, ERi]
    alpha_list.append(alpha)
    beta_list.append(beta)
    ERi_list.append(ERi)
final_output_df= pd.DataFrame.from_dict(final_output).T
final_output_df.columns = ['alpha',' beta', 'ERi']

# SML

In [7]:
final_output_df

,alpha,beta,ERi
ETHUSDT,0.0022,1.0686,1.996551
BNBUSDT,0.0032,1.0428,2.332734
NEOUSDT,-0.0002,1.0929,1.147692
LTCUSDT,-0.0005,1.1379,1.088454
QTUMUSDT,0.0011,1.1634,1.700936
ADAUSDT,0.0035,1.0041,2.399009
XRPUSDT,0.0010,0.9753,1.454341
EOSUSDT,-0.0014,1.1332,0.754704
ONEUSDT,0.0055,1.1071,3.244052
IOTAUSDT,0.0011,1.1085,1.639616


2022-05-03 16:18:38.018 python[38030:2423096] _TIPropertyValueIsValid called with 4 on nil context!
2022-05-03 16:18:38.018 python[38030:2423096] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2022-05-03 16:18:38.018 python[38030:2423096] Text input context does not respond to _valueForTIProperty:
2022-05-03 16:18:38.020 python[38030:2423096] _TIPropertyValueIsValid called with 4 on nil context!
2022-05-03 16:18:38.020 python[38030:2423096] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2022-05-03 16:18:38.020 python[38030:2423096] Text input context does not respond to _valueForTIProperty:
2022-05-03 16:18:38.030 python[38030:2423096] _TIPropertyValueIsValid called with 4 on nil context!
2022-05-03 16:18:38.030 python[38030:2423096] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2022-05-03 16:18:38.030 python[38030:2423096] Text input context does not respond to _valueF

In [14]:
%matplotlib qt

In [15]:
# font_pro = FontProperties(fname='C:\\Windows\\Fonts\\mingliu.ttc')
fig, ax = plt.subplots(figsize=(15, 10))
fig.suptitle('Single Index Model — Crypto market', fontsize=20, fontweight='bold')
fig.subplots_adjust(top=0.88)
ax.set_title('2020/01/01–2022/03/01', fontsize=14)
ax.scatter(beta_list, ERi_list, c = '#047cbd')
 
for k, v in final_output.items():
    ax.annotate(k, (float(v[1]), float(v[-1])))
 
 
ax.axhline(y=0, color='k', ls='-', alpha = 0.5)
ax.axvline(x=0, color='k', ls='-', alpha = 0.5)
 
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
 
x1, x2 = 0, 2
y1, y2 = 0, ret_df['BTCUSDT'].mean()*365*2
 
plt.xticks(np.arange(-0.4, 2.1, 0.4))
plt.plot([x1, x2], [y1, y2], 'k--', linewidth=2)
plt.show()